In [2]:
import torch
from torchinfo import summary
import coremltools as ct
import numpy as np
from PIL import Image
import os
from PIL import Image


## Загрузка и анализ модели YOLOv5

### Загрузка предобученной модели

Здесь мы загружаем предобученную модель YOLOv5 с помощью `torch.hub.load`. Модель 'yolov5s', самая легкая версия YOLOv5, выбрана для оптимального сочетания производительности и скорости обработки. Это делает ее подходящей для задач обнаружения объектов в реальном времени на устройствах с ограниченными ресурсами.

### Суммарное описание модели

Функция `summary` из библиотеки Torch предоставляет детальный обзор архитектуры модели, включая количество слоев, параметров и формы входных/выходных данных каждого слоя. Это помогает понять структуру модели и облегчает диагностику при разработке и настройке приложений на ее основе.


In [3]:
model =  torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
summary(model)

Using cache found in /Users/egorlogutov/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-4-22 Python-3.11.9 torch-2.2.2 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Layer (type:depth-idx)                                  Param #
AutoShape                                               --
├─DetectMultiBackend: 1-1                               --
│    └─DetectionModel: 2-1                              --
│    │    └─Sequential: 3-1                             (7,225,885)
Total params: 7,225,885
Trainable params: 0
Non-trainable params: 7,225,885

## Переключение модели в режим оценки

После загрузки модели, вызываем `model.eval()` для перевода модели в режим оценки. Это важный шаг перед использованием модели для инференса, так как он деактивирует слои, такие как Dropout и Batch Normalization, которые используются во время обучения для улучшения процесса обучения, но не нужны при применении модели. В режиме оценки модель будет вести себя детерминированно, обеспечивая стабильность результатов при обнаружении объектов.


In [4]:
model.eval()

AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

## Подготовка изображения для обработки моделью

### Загрузка и изменение размера изображения

Изображение загружается с помощью `Image.open` из библиотеки PIL и изменяется до размера 640x640 пикселей. Это обеспечивает совместимость с ожидаемыми входными размерами модели YOLOv5.

### Преобразование в массив и нормализация

Загруженное изображение преобразуется в массив numpy с помощью `np.array`. Затем значения пикселей нормализуются путем деления на 255.0, чтобы масштабировать их в диапазон от 0 до 1, что является стандартной практикой перед подачей данных в нейросеть.

### Перестановка осей и добавление размерности

Если изображение содержит три канала (RGB), оси переставляются так, чтобы каналы цвета находились первыми (`np.transpose(image_array, (2, 0, 1))`). Это необходимо, так как модель YOLO ожидает, что первая размерность тензора будет представлять каналы цвета. После этого, к массиву добавляется дополнительная размерность (`np.expand_dims`), чтобы подготовить его к подаче в модель как пакет данных размером 1 (batch size).


In [13]:
current_directory = os.getcwd()
parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))

image = Image.open(f'{parent_directory}/Files/qr.webp')
image = image.resize((640, 640))

image_array = np.array(image)

image_array = image_array / 255.0

image_array = image_array.astype(np.float32)

if image_array.shape[2] == 3:  # если последняя ось - это каналы
    image_array = np.transpose(image_array, (2, 0, 1))
value_input = np.expand_dims(image_array, axis=0)


## Получение предсказаний от модели YOLOv5

### Подача данных в модель

Подготовленное изображение загружается в тензор PyTorch с помощью `torch.tensor(image_array)` и сохраняется в переменной `input_data`. Это изображение будет использоваться для генерации предсказаний модели.

### Вычисление предсказаний

С помощью `torch.no_grad()` мы отключаем вычисление градиентов для ускорения процесса предсказания. Затем вызываем модель, передавая в нее подготовленное изображение `input_data`. Метод `model` автоматически применяет модель к входным данным и возвращает результаты предсказаний, которые сохраняются в переменной `predictions`.

Этот процесс демонстрирует применение модели YOLOv5 для обработки изображений и получения предсказаний об объектах, обнаруженных на изображении.


In [6]:
input_data = torch.tensor(image_array)

with torch.no_grad():
    predictions = model(input_data)


## Сохранение предсказаний модели в файл и проверка их формы

### Сохранение предсказаний

Предсказания модели, полученные в предыдущем шаге, сохраняются в текстовый файл `YOLO_значение.txt` с использованием функции `np.savetxt`. Эта функция принимает на вход путь к файлу, данные и разделитель. Перед сохранением данные предсказаний преобразуются в одномерный массив с помощью метода `flatten()`, чтобы облегчить запись в текстовом формате.

### Проверка формы предсказаний

Команда `predictions.shape` используется для вывода формы тензора предсказаний. Это позволяет убедиться в том, что размерность данных соответствует ожиданиям и что модель возвращает корректные выходные данные. Знание формы предсказаний важно для дальнейшей обработки и анализа результатов.


In [7]:
file_path = f"{parent_directory}/Files/YOLO_значение.txt"

np.savetxt(file_path, predictions.flatten(), delimiter=', ')

predictions.shape


torch.Size([1, 25200, 85])

Мы получили результаты относительно которых сможем проверить правильно ли сконвертировалась модель
Далее перейдем к сохранению модели и конвертации в mlmodel формат

## Трассировка и сохранение модели YOLOv5 в формате TorchScript

### Подготовка примера входных данных для трассировки

Создаем тензор `input_trace_example`, который представляет собой батч из одного RGB изображения размером 640x640 пикселей. Изображение нормализуется путем деления каждого пикселя на 255.0, что приводит значения к диапазону [0, 1], подходящему для входа в нейронную сеть.

### Трассировка модели

Используем `torch.jit.trace` для создания трассированной версии модели YOLOv5. Этот метод выполняет модель с указанным примером входных данных, автоматически генерируя версию модели, которая может быть эффективно выполняема без динамических вычислений Python. Трассировка полезна для ускорения инференса и упрощения развертывания модели на различных платформах.

### Сохранение трассированной модели

Трассированная модель сохраняется в файл `YOLO_model_traced.pt` с использованием `torch.jit.save`. Файл TorchScript, полученный в результате, является самодостаточным и может быть загружен и использован на платформах без установленного Python или зависимостей PyTorch.

Этот процесс обеспечивает удобное и быстрое развертывание модели в продакшн, позволяя использовать ее в высокопроизводительных приложениях для обработки изображений.


In [8]:
input_trase_example = (torch.randn(1, 3, 640, 640) / 255.0)  # Батч из 4 RGB изображений 640x640 с нормализацией значений
traced_model = torch.jit.trace(model, input_trase_example)
torch.jit.save(traced_model, f'{parent_directory}/Files/YOLO_model_traced.pt')

/Users/egorlogutov/.cache/torch/hub/ultralytics_yolov5_master/models/yolo.py:100: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.dynamic or self.grid[i].shape[2:4] != x[i].shape[2:4]:
/Users/egorlogutov/.cache/torch/hub/ultralytics_yolov5_master/models/yolo.py:100: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.dynamic or self.grid[i].shape[2:4] != x[i].shape[2:4]:


In [9]:
YOLO_loaded_model = torch.jit.load(f'{parent_directory}/Files/YOLO_model_traced.pt')
YOLO_loaded_model.eval()

RecursiveScriptModule(
  original_name=AutoShape
  (model): RecursiveScriptModule(
    original_name=DetectMultiBackend
    (model): RecursiveScriptModule(
      original_name=DetectionModel
      (model): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(
          original_name=Conv
          (conv): RecursiveScriptModule(original_name=Conv2d)
          (act): RecursiveScriptModule(original_name=SiLU)
        )
        (1): RecursiveScriptModule(
          original_name=Conv
          (conv): RecursiveScriptModule(original_name=Conv2d)
          (act): RecursiveScriptModule(original_name=SiLU)
        )
        (2): RecursiveScriptModule(
          original_name=C3
          (cv1): RecursiveScriptModule(
            original_name=Conv
            (conv): RecursiveScriptModule(original_name=Conv2d)
            (act): RecursiveScriptModule(original_name=SiLU)
          )
          (cv2): RecursiveScriptModule(
            original_name=Conv
    

In [10]:
summary(YOLO_loaded_model)

Layer (type:depth-idx)                                  Param #
AutoShape                                               --
├─DetectMultiBackend: 1-1                               --
│    └─DetectionModel: 2-1                              --
│    │    └─Sequential: 3-1                             (7,225,885)
Total params: 7,225,885
Trainable params: 0
Non-trainable params: 7,225,885

In [11]:
YOLO_example_input = (torch.randn(1, 3, 640, 640) / 255.0)

In [12]:
mlmodel = ct.convert(
    YOLO_loaded_model,
    inputs=[ct.TensorType(shape=YOLO_example_input.shape)],
    convert_to='neuralnetwork'
)

'''Можно запустить модель для проверки перед сохранением'''


# Сохраняем модель в файл
mlmodel.save(f"{parent_directory}/Files/YOLO_CoreML.mlmodel")

Support for converting Torch Script Models is experimental. If possible you should use a traced model for conversion.
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 610/610 [00:04<00:00, 133.93 ops/s] 


In [15]:
YOLO_coreML = ct.models.MLModel(f'{parent_directory}/Files/YOLO_CoreML.mlmodel')

In [16]:
spec = YOLO_coreML.get_spec()
print(spec.description.input)
print(spec.description.output)

[name: "ims_1"
type {
  multiArrayType {
    shape: 1
    shape: 3
    shape: 640
    shape: 640
    dataType: FLOAT32
  }
}
]
[name: "var_871"
type {
  multiArrayType {
    dataType: FLOAT32
  }
}
]


In [17]:
input_data = {"ims_1": value_input}

YOLO_predict = YOLO_coreML.predict(input_data)

In [18]:
b = YOLO_predict['var_871'].flatten()

YOLO_data = np.loadtxt(f'{parent_directory}/Files/YOLO_значение.txt')


print(f"Предсказание PyTorch - {YOLO_data}")
print(f"Предсказание CoreML - {b}")



Предсказание PyTorch - [     5.8368      4.7197      12.454 ...   0.0074723   0.0019991   0.0073827]
Предсказание CoreML - [     5.8125      4.7188      12.391 ...   0.0075684   0.0019531   0.0073242]
